In [9]:
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [3]:
base = pd.read_csv('/workspaces/learning-machine-learning/udemy/a_z/bolsa_valores/petr4_treinamento.csv')
base = base.dropna()
base_treinamento = base.iloc[:, 1:7].values

In [5]:
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [15]:
normalizador_previsao = MinMaxScaler(feature_range=(0,1))
normalizador_previsao.fit_transform(base_treinamento[:,0:1])

array([[0.76501938],
       [0.7562984 ],
       [0.78149225],
       ...,
       [0.57122093],
       [0.57655039],
       [0.57655039]])

In [7]:
previsores = []
preco_real = []
for i in range(90, 1242):
    previsores.append(base_treinamento_normalizada[i-90:i, 0:6])
    preco_real.append(base_treinamento_normalizada[i, 0])
previsores, preco_real = np.array(previsores), np.array(preco_real)

In [8]:
regressor = Sequential()
regressor.add(LSTM(units=100, return_sequences=True, input_shape=(previsores.shape[1], 6)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units=1, activation='sigmoid'))

regressor.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [10]:
es = EarlyStopping(monitor='loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, verbose=1)
mcp = ModelCheckpoint(filepath='pesos.h5', monitor='loss', save_best_only=True, verbose=1)


In [11]:
regressor.fit(previsores, preco_real, epochs=100, batch_size=32, callbacks=[es, rlr, mcp])

Epoch 1/100
36/36 [==============================] - ETA: 0s - loss: 0.0180 - mean_absolute_error: 0.1040
Epoch 1: loss improved from inf to 0.01799, saving model to pesos.h5
36/36 [==============================] - 8s 113ms/step - loss: 0.0180 - mean_absolute_error: 0.1040 - lr: 0.0010
Epoch 2/100
 1/36 [..............................] - ETA: 3s - loss: 0.0099 - mean_absolute_error: 0.0844

/usr/local/python/3.11.5/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


36/36 [==============================] - ETA: 0s - loss: 0.0078 - mean_absolute_error: 0.0699
Epoch 2: loss improved from 0.01799 to 0.00783, saving model to pesos.h5
36/36 [==============================] - 4s 117ms/step - loss: 0.0078 - mean_absolute_error: 0.0699 - lr: 0.0010
Epoch 3/100
36/36 [==============================] - ETA: 0s - loss: 0.0059 - mean_absolute_error: 0.0606
Epoch 3: loss improved from 0.00783 to 0.00590, saving model to pesos.h5
36/36 [==============================] - 4s 116ms/step - loss: 0.0059 - mean_absolute_error: 0.0606 - lr: 0.0010
Epoch 4/100
36/36 [==============================] - ETA: 0s - loss: 0.0059 - mean_absolute_error: 0.0600
Epoch 4: loss improved from 0.00590 to 0.00585, saving model to pesos.h5
36/36 [==============================] - 4s 115ms/step - loss: 0.0059 - mean_absolute_error: 0.0600 - lr: 0.0010
Epoch 5/100
36/36 [==============================] - ETA: 0s - loss: 0.0047 - mean_absolute_error: 0.0528
Epoch 5: loss improved from 0.

In [12]:
base_teste = pd.read_csv('/workspaces/learning-machine-learning/udemy/a_z/bolsa_valores/petr4_teste.csv')
preco_real_teste = base_teste.iloc[:, 1:2].values
frames = [base, base_teste]
base_completa = pd.concat(frames)
base_completa = base_completa.drop('Date', axis=1)

In [13]:
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values
entradas = normalizador.transform(entradas)

In [14]:
x_teste = []
for i in range(90, 112):
    x_teste.append(entradas[i-90:i, 0:6])
x_teste = np.array(x_teste)


In [16]:
previsoes = regressor.predict(x_teste)
previsoes = normalizador_previsao.inverse_transform(previsoes)

1/1 [==============================] - 1s 1s/step


In [17]:
previsoes.mean()

17.46247